# Training a model for catchment classification.
* kategoryzacja zlewni oparta o wielkość zlewni i wyniki oblczeń zlewni
* opad nie jest przyjmowany ponieważ jest tylko jedna wartość i jego wartość jest standaryzowana do zera

# Imports

In [1]:
import swmmio
import pyswmm
import pandas as pd
import numpy as np
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

from plotly.subplots import make_subplots


desired_width = 500
pd.set_option("display.width", desired_width)
np.set_printoptions(linewidth=desired_width)
pd.set_option("display.max_columns", 30)

## Catchment categories

In [2]:
classes = pd.DataFrame(
    data={
        "classes": [
            "compact_urban_development",
            "urban",
            "loose_urban_development",
            "wooded_area",
            "grassy",
            "loose_soil",
            "steep_area",
        ]
    }
)
classes

,classes
0,compact_urban_development
1,urban
2,loose_urban_development
3,wooded_area
4,grassy
5,loose_soil
6,steep_area


## Get files

In [3]:
INP_FILE = "dataset/subcatchment_dataset.inp"
RPT_FILE = "dataset/subcatchment_dataset.inp"

## Run simulation

In [4]:
with pyswmm.Simulation(INP_FILE) as sim:
    for step in sim:
        pass

## Read inp and rpt file as swmmio model object

In [5]:
model = swmmio.Model(INP_FILE)

## Get subcatchments data from the model

In [6]:
raw_subcatchments = model.subcatchments.dataframe
subcatchments = raw_subcatchments.copy()
subcatchments[:5]

,Raingage,Outlet,Area,PercImperv,Width,PercSlope,CurbLength,N-Imperv,N-Perv,S-Imperv,S-Perv,PctZero,RouteTo,TotalPrecip,TotalRunon,TotalEvap,TotalInfil,ImpervRunoff,PervRunoff,TotalRunoffIn,TotalRunoffMG,PeakRunoff,RunoffCoeff,coords
Name,,,,,,,,,,,,,,,,,,,,,,,,
S1,Raingage2,O1,0.50,20.00,300.00,5.00,0,0.150,0.41,1.27,5.08,70,OUTLET,10.10,0.00,0.00,5.91,1.95,2.17,4.12,0.02,0.02,0.407,"[(777180.0, 592590.0), (777180.0, 592585.0), (777175.0, 592585.0), (777175.0, 592590.0), (777180.0, 592590.0)]"
S10,Raingage2,O1,1.87,45.00,136.75,15.00,0,0.013,0.15,1.27,5.08,90,OUTLET,10.10,0.00,0.00,4.13,4.50,1.43,5.93,0.11,0.09,0.587,NaN
S100,Raingage2,O1,1.71,45.00,130.77,10.00,0,0.013,0.15,1.27,5.08,90,OUTLET,10.10,0.00,0.00,4.18,4.50,1.38,5.87,0.10,0.08,0.581,NaN
S1000,Raingage2,O1,0.68,10.00,82.46,5.09,0,0.150,0.41,1.27,5.08,10,OUTLET,10.10,0.00,0.00,7.73,0.90,1.36,2.26,0.02,0.01,0.224,NaN
S1001,Raingage2,O1,0.26,83.33,50.99,45.00,0,0.013,0.05,1.27,5.08,80,OUTLET,10.10,0.00,0.00,1.04,8.23,0.65,8.88,0.02,0.02,0.879,NaN


## Drop unused columns

In [7]:
subcatchments.columns

Index(['Raingage', 'Outlet', 'Area', 'PercImperv', 'Width', 'PercSlope', 'CurbLength', 'N-Imperv', 'N-Perv', 'S-Imperv', 'S-Perv', 'PctZero', 'RouteTo', 'TotalPrecip', 'TotalRunon', 'TotalEvap', 'TotalInfil', 'ImpervRunoff', 'PervRunoff', 'TotalRunoffIn', 'TotalRunoffMG', 'PeakRunoff', 'RunoffCoeff', 'coords'], dtype='object')

In [8]:
subcatchments.drop(['Raingage',  'Outlet', 'PercImperv', 'Width', 'PercSlope', 'CurbLength', 'N-Imperv', 'N-Perv', 'S-Imperv', 'S-Perv', 'PctZero', 'RouteTo', 'coords'], axis=1, inplace=True)
subcatchments

,Area,TotalPrecip,TotalRunon,TotalEvap,TotalInfil,ImpervRunoff,PervRunoff,TotalRunoffIn,TotalRunoffMG,PeakRunoff,RunoffCoeff
Name,,,,,,,,,,,
S1,0.50,10.10,0.00,0.00,5.91,1.95,2.17,4.12,0.02,0.02,0.407
S10,1.87,10.10,0.00,0.00,4.13,4.50,1.43,5.93,0.11,0.09,0.587
S100,1.71,10.10,0.00,0.00,4.18,4.50,1.38,5.87,0.10,0.08,0.581
S1000,0.68,10.10,0.00,0.00,7.73,0.90,1.36,2.26,0.02,0.01,0.224
S1001,0.26,10.10,0.00,0.00,1.04,8.23,0.65,8.88,0.02,0.02,0.879
...,...,...,...,...,...,...,...,...,...,...,...
S995,1.10,10.10,0.00,0.00,7.53,0.90,1.56,2.46,0.03,0.02,0.243
S996,1.89,10.10,0.00,0.00,7.55,0.46,2.04,2.49,0.05,0.03,0.247
S997,1.19,10.10,0.00,0.00,7.91,0.90,1.18,2.08,0.02,0.01,0.206


## Get categories

In [9]:
# print([attr for attr in dir(model.inp) if not attr.startswith("_")])
categories = model.inp.tags
categories

,Name,Tag
ElementType,,
Subcatch,S1,#comment
Subcatch,S2,compact_urban_development
Subcatch,S3,loose_soil
Subcatch,S4,wooded_area
Subcatch,S5,loose_urban_development
...,...,...
Subcatch,S997,grassy
Subcatch,S998,steep_area
Subcatch,S999,urban


### Add categories column to subcatchments DataFrame

In [10]:
# Reset the index of subcatchments DataFrame
subcatchments.reset_index(inplace=True)

# Merge the two DataFrames on the "Name" column
merged_df = subcatchments.merge(model.inp.tags, left_on="Name", right_on="Name", how="left")

# Rename the 'Tag' column to 'categories'
merged_df.rename(columns={"Tag": "categories"}, inplace=True)

# Set the index back to "Name"
merged_df.set_index("Name", inplace=True)

In [11]:
merged_df[:3]

,Area,TotalPrecip,TotalRunon,TotalEvap,TotalInfil,ImpervRunoff,PervRunoff,TotalRunoffIn,TotalRunoffMG,PeakRunoff,RunoffCoeff,categories
Name,,,,,,,,,,,,
S1,0.50,10.10,0.00,0.00,5.91,1.95,2.17,4.12,0.02,0.02,0.407,#comment
S10,1.87,10.10,0.00,0.00,4.13,4.50,1.43,5.93,0.11,0.09,0.587,compact_urban_development
S100,1.71,10.10,0.00,0.00,4.18,4.50,1.38,5.87,0.10,0.08,0.581,compact_urban_development


# Split data into features and target

In [12]:
X = merged_df.drop("categories", axis=1)
y = merged_df["categories"]

print(X.shape, y.shape)

(1001, 11) (1001,)


In [13]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1001 entries, S1 to S999
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Area           1001 non-null   float64
 1   TotalPrecip    1001 non-null   object 
 2   TotalRunon     1001 non-null   object 
 3   TotalEvap      1001 non-null   object 
 4   TotalInfil     1001 non-null   object 
 5   ImpervRunoff   1001 non-null   float64
 6   PervRunoff     1001 non-null   object 
 7   TotalRunoffIn  1001 non-null   float64
 8   TotalRunoffMG  1001 non-null   float64
 9   PeakRunoff     1001 non-null   float64
 10  RunoffCoeff    1001 non-null   float64
dtypes: float64(6), object(5)
memory usage: 93.8+ KB


In [14]:
X["TotalPrecip"] = pd.to_numeric(X["TotalPrecip"])
X["TotalRunon"] = pd.to_numeric(X["TotalRunon"])
X["TotalEvap"] = pd.to_numeric(X["TotalEvap"])
X["TotalInfil"] = pd.to_numeric(X["TotalInfil"])
X["PervRunoff"] = pd.to_numeric(X["PervRunoff"])

In [15]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1001 entries, S1 to S999
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Area           1001 non-null   float64
 1   TotalPrecip    1001 non-null   float64
 2   TotalRunon     1001 non-null   float64
 3   TotalEvap      1001 non-null   float64
 4   TotalInfil     1001 non-null   float64
 5   ImpervRunoff   1001 non-null   float64
 6   PervRunoff     1001 non-null   float64
 7   TotalRunoffIn  1001 non-null   float64
 8   TotalRunoffMG  1001 non-null   float64
 9   PeakRunoff     1001 non-null   float64
 10  RunoffCoeff    1001 non-null   float64
dtypes: float64(11)
memory usage: 93.8+ KB


In [16]:
X = pd.get_dummies(X, drop_first=True)

In [17]:
X.head()

,Area,TotalPrecip,TotalRunon,TotalEvap,TotalInfil,ImpervRunoff,PervRunoff,TotalRunoffIn,TotalRunoffMG,PeakRunoff,RunoffCoeff
Name,,,,,,,,,,,
S1,0.50,10.1,0.0,0.0,5.91,1.95,2.17,4.12,0.02,0.02,0.407
S10,1.87,10.1,0.0,0.0,4.13,4.50,1.43,5.93,0.11,0.09,0.587
S100,1.71,10.1,0.0,0.0,4.18,4.50,1.38,5.87,0.10,0.08,0.581
S1000,0.68,10.1,0.0,0.0,7.73,0.90,1.36,2.26,0.02,0.01,0.224
S1001,0.26,10.1,0.0,0.0,1.04,8.23,0.65,8.88,0.02,0.02,0.879


## Data standarization

In [18]:
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X)
X_normalized_df = pd.DataFrame(X_normalized, columns=X.columns, index=X.index)

In [19]:
X_normalized_df.head()

,Area,TotalPrecip,TotalRunon,TotalEvap,TotalInfil,ImpervRunoff,PervRunoff,TotalRunoffIn,TotalRunoffMG,PeakRunoff,RunoffCoeff
Name,,,,,,,,,,,
S1,0.246231,0.0,0.0,0.0,0.608479,0.216687,0.683871,0.392630,0.111111,0.142857,0.391528
S10,0.934673,0.0,0.0,0.0,0.386534,0.534247,0.445161,0.622618,0.611111,0.642857,0.622593
S100,0.854271,0.0,0.0,0.0,0.392768,0.534247,0.429032,0.614994,0.555556,0.571429,0.614891
S1000,0.336683,0.0,0.0,0.0,0.835411,0.085928,0.422581,0.156290,0.111111,0.071429,0.156611
S1001,0.125628,0.0,0.0,0.0,0.001247,0.998755,0.193548,0.997459,0.111111,0.142857,0.997433


## Encode target labels

In [20]:
encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y.values.reshape(-1, 1))

In [21]:
y_encoded

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

## Split the data into training and testing sets

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X_normalized_df, y_encoded, test_size=0.2, random_state=42)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (800, 11)
y_train shape: (800, 8)
X_test shape: (201, 11)
y_test shape: (201, 8)


# Build the model

In [23]:
model = Sequential()
model.add(Dense(16, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.2))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=y_train.shape[1], activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                192       
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 8)                 136       
                                                                 
 dropout_1 (Dropout)         (None, 8)                 0         
                                                                 
 dense_2 (Dense)             (None, 8)                 72        
                                                                 
Total params: 400
Trainable params: 400
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [25]:
history = model.fit(
    x=X_train,
    y=y_train,
    epochs=150,
    validation_data=(X_test, y_test),
    validation_split=0.2,
    verbose=1,
    batch_size=16,
    callbacks=[EarlyStopping(monitor='val_loss', patience=10)]
)

Epoch 1/150
50/50 [==============================] - 1s 7ms/step - loss: 2.0472 - accuracy: 0.2125 - val_loss: 2.0031 - val_accuracy: 0.3781
Epoch 2/150
50/50 [==============================] - 0s 4ms/step - loss: 1.9631 - accuracy: 0.3750 - val_loss: 1.8885 - val_accuracy: 0.4080
Epoch 3/150
50/50 [==============================] - 0s 2ms/step - loss: 1.8161 - accuracy: 0.4112 - val_loss: 1.7354 - val_accuracy: 0.3831
Epoch 4/150
50/50 [==============================] - 0s 3ms/step - loss: 1.6991 - accuracy: 0.4263 - val_loss: 1.6226 - val_accuracy: 0.4478
Epoch 5/150
50/50 [==============================] - 0s 2ms/step - loss: 1.5968 - accuracy: 0.4525 - val_loss: 1.5350 - val_accuracy: 0.4527
Epoch 6/150
50/50 [==============================] - 0s 2ms/step - loss: 1.5368 - accuracy: 0.4538 - val_loss: 1.4628 - val_accuracy: 0.4378
Epoch 7/150
50/50 [==============================] - 0s 2ms/step - loss: 1.4704 - accuracy: 0.4613 - val_loss: 1.4065 - val_accuracy: 0.4378
Epoch 8/150
5

In [26]:
# Evaluate ANN model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test set accuracy: {accuracy:.2f}")

7/7 [==============================] - 0s 1ms/step - loss: 0.1403 - accuracy: 0.9950
Test set accuracy: 1.00


In [27]:
y_pred = model.predict(X_test)

7/7 [==============================] - 0s 1ms/step


In [28]:
metrics = pd.DataFrame(history.history)
metrics['epoch'] = history.epoch
metrics

,loss,accuracy,val_loss,val_accuracy,epoch
0,2.047151,0.21250,2.003150,0.378109,0
1,1.963096,0.37500,1.888466,0.407960,1
2,1.816067,0.41125,1.735420,0.383085,2
3,1.699059,0.42625,1.622568,0.447761,3
4,1.596844,0.45250,1.534990,0.452736,4
...,...,...,...,...,...
145,0.379124,0.83875,0.145419,0.980099,145
146,0.415525,0.82750,0.151462,0.995025,146
147,0.375594,0.85125,0.150578,0.995025,147
148,0.379353,0.83375,0.144962,0.990050,148


# Model evaluate

In [29]:
fig = make_subplots(rows=1, cols=2)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['accuracy'], name='accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['loss'], name='loss'), row=1, col=2)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['val_accuracy'], name='val_accuracy'), row=1, col=1)
fig.add_trace(go.Scatter(x=metrics['epoch'], y=metrics['val_loss'], name='val_loss'), row=1, col=2)

fig.update_xaxes(title_text='epochs')
fig.update_yaxes(title_text='accuracy')
fig.update_layout(width=1000, title='Accuracy and Loss')
fig.show()

In [30]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(test_acc)

0.9950248599052429


Predykcja na podstawie modelu.
* model.evaluate(y_true, y_pred) - pozwala obliczyć metryki modelu
* model.predict_classes() - pozwala zwrócić odpowiednio przewidziane klasy
* model.predict_proba(), model.predict() - pozwala zwrócić prawdopodobieństwo danej klasy

In [31]:
predictions = model.predict(X_test)
predictions

7/7 [==============================] - 0s 1ms/step


array([[1.69264984e-11, 2.17485867e-17, 9.99910831e-01, ..., 1.07301265e-17, 2.58373132e-11, 6.48991627e-05],
       [8.99454986e-04, 1.07938668e-03, 9.21731666e-02, ..., 1.58882285e-05, 1.30792230e-01, 4.99303862e-02],
       [2.64391797e-08, 6.92545093e-21, 9.96708393e-01, ..., 4.50949298e-18, 6.02542238e-10, 3.45617025e-07],
       ...,
       [1.85373960e-12, 3.27888847e-04, 2.20580846e-02, ..., 2.20225033e-10, 3.70742548e-08, 9.77613032e-01],
       [2.22744643e-06, 8.28685500e-20, 9.34113085e-01, ..., 3.66637947e-16, 1.92637284e-08, 2.76250489e-07],
       [3.35217780e-03, 2.32938888e-07, 1.76691413e-02, ..., 8.93883225e-07, 3.38527039e-02, 5.06017474e-04]], dtype=float32)

In [32]:
predictions_df = pd.DataFrame(predictions)
predictions_df

,0,1,2,3,4,5,6,7
0,1.692650e-11,2.174859e-17,0.999911,1.333412e-07,2.402350e-05,1.073013e-17,2.583731e-11,6.489916e-05
1,8.994550e-04,1.079387e-03,0.092173,5.164032e-04,7.245930e-01,1.588823e-05,1.307922e-01,4.993039e-02
2,2.643918e-08,6.925451e-21,0.996708,1.184879e-03,2.106340e-03,4.509493e-18,6.025422e-10,3.456170e-07
3,2.777517e-08,1.642078e-20,0.996775,9.924450e-04,2.232034e-03,7.235224e-18,8.478009e-10,4.833166e-07
4,4.159383e-07,2.431866e-16,0.989752,1.432915e-03,8.802564e-03,9.449944e-15,7.679556e-08,1.169400e-05
...,...,...,...,...,...,...,...,...
196,1.223195e-11,5.169967e-04,0.031142,2.069426e-11,3.050899e-06,7.859232e-10,1.471532e-07,9.683378e-01
197,3.042107e-06,1.668968e-20,0.885388,1.010989e-01,1.351037e-02,2.541792e-16,1.127028e-08,1.268463e-07
198,1.853740e-12,3.278888e-04,0.022058,3.148462e-12,8.711029e-07,2.202250e-10,3.707425e-08,9.776130e-01
199,2.227446e-06,8.286855e-20,0.934113,4.994800e-02,1.593647e-02,3.666379e-16,1.926373e-08,2.762505e-07


In [33]:
predictions_cls = predictions.argmax(axis=-1)
predictions_cls

array([2, 4, 2, 2, 2, 1, 2, 7, 3, 2, 3, 7, 2, 5, 5, 1, 5, 4, 2, 3, 7, 3, 7, 4, 7, 1, 2, 4, 6, 1, 2, 6, 6, 4, 6, 1, 7, 2, 7, 4, 2, 5, 7, 2, 2, 2, 7, 2, 3, 4, 1, 1, 4, 1, 3, 2, 7, 2, 7, 3, 2, 7, 2, 6, 7, 2, 3, 1, 3, 2, 2, 2, 4, 2, 2, 2, 1, 2, 2, 1, 7, 2, 2, 2, 6, 2, 5, 7, 2, 7, 6, 2, 3, 4, 2, 2, 2, 2, 6, 6, 3, 2, 6, 4, 5, 3, 6, 3, 2, 2, 1, 2, 2, 2, 6, 7, 3, 1, 2, 2, 6, 7, 2, 2, 6, 3, 2, 2, 3, 5, 4, 1, 6, 2, 1, 4, 7, 3, 3, 7, 6, 2, 7, 2, 2, 2, 4, 2, 4, 2, 3, 5, 2, 6, 1, 2, 5, 2, 4, 3, 4, 6, 2, 3,
       6, 5, 2, 1, 4, 2, 6, 2, 1, 5, 1, 2, 4, 2, 3, 2, 7, 1, 2, 3, 5, 6, 6, 1, 4, 2, 2, 2, 7, 7, 2, 1, 7, 2, 7, 2, 4], dtype=int64)

# Zapisanie modelu

In [35]:
model.save(r'C:\Users\Dell\Documents\Git\stormwater-analysis\stormwater_analysis\data\area_rpt')

INFO:tensorflow:Assets written to: C:\Users\Dell\Documents\Git\stormwater-analysis\stormwater_analysis\data\area_rpt\assets


# Załadowanie modelu

In [36]:
from tensorflow import keras
model = keras.models.load_model(r'C:\Users\Dell\Documents\Git\stormwater-analysis\stormwater_analysis\data\area_rpt')

In [37]:
pred = model.predict(X_test)
pred

7/7 [==============================] - 0s 2ms/step


array([[1.69264984e-11, 2.17485867e-17, 9.99910831e-01, ..., 1.07301265e-17, 2.58373132e-11, 6.48991627e-05],
       [8.99454986e-04, 1.07938668e-03, 9.21731666e-02, ..., 1.58882285e-05, 1.30792230e-01, 4.99303862e-02],
       [2.64391797e-08, 6.92545093e-21, 9.96708393e-01, ..., 4.50949298e-18, 6.02542238e-10, 3.45617025e-07],
       ...,
       [1.85373960e-12, 3.27888847e-04, 2.20580846e-02, ..., 2.20225033e-10, 3.70742548e-08, 9.77613032e-01],
       [2.22744643e-06, 8.28685500e-20, 9.34113085e-01, ..., 3.66637947e-16, 1.92637284e-08, 2.76250489e-07],
       [3.35217780e-03, 2.32938888e-07, 1.76691413e-02, ..., 8.93883225e-07, 3.38527039e-02, 5.06017474e-04]], dtype=float32)